In [1]:
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize

csv_filename = 'product_details_processor.csv'
csv_bought_together = 'items_bought_together_processor.csv'
csv_customer_reviews = 'links_to_customer_reviews_processor.csv'
txt_customer_reviews = 'links_to_customer_reviews_processor.txt'

In [2]:
df = pd.read_json('processor_product_output.jsonl')
df.head()

,name,product_summary,product_tech_spec,product_addl_info,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews
0,"AMD Ryzen 5 3600 6-Core, 12-Thread Unlocked De...",None,"[{'info': 'Brand', 'value': 'AMD'}, {'info': '...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/AMD-Ryzen-3600-12-Thread-Processor/product-re...
1,Intel Core i9-9900K Desktop Processor 8 Cores ...,None,"[{'info': 'Brand', 'value': 'Intel'}, {'info':...","[{'info': 'Customer Reviews', 'value': '/* * F...",Ideabuy,https://www.amazon.com/gp/help/seller/at-a-gla...,None,None,/Intel-i9-9900K-Processor-Unlocked-BX806849900...
2,"AMD Ryzen 9 3900X 12-core, 24-thread unlocked ...",None,"[{'info': 'Brand', 'value': 'AMD'}, {'info': '...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/AMD-Ryzen-3900X-24-Thread-Processor/product-r...
3,"AMD Ryzen 7 3700X 8-Core, 16-Thread Unlocked D...",None,"[{'info': 'Brand', 'value': 'AMD'}, {'info': '...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/AMD-Ryzen-3700X-16-Thread-Processor/product-r...
4,Intel Core i5-10400 Desktop Processor 6 Cores ...,None,"[{'info': 'Brand', 'value': 'Intel'}, {'info':...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/Intel-i5-10400-Desktop-Processor-LGA1200/prod...


In [3]:
df.name = df.name.str.replace('"','inch')

In [4]:
df = df.dropna(axis=0, how='all')

In [5]:
df.link_to_all_reviews = 'https://www.amazon.com/' + df.link_to_all_reviews

In [6]:
df.reset_index(drop=True, inplace = True)

In [7]:
# Extraction of all specifications in nested JSON-dicts to be used for building additional columns 
  
new_columns = []
for i in range(len(df)):
    if df.product_summary[i] is not None:
        product_sum = df.product_summary[i]
        x = len(product_sum)  
        for j in range(x):
            new_columns.append(product_sum[j]['info'])
    if df.product_tech_spec[i] is not None:
        product_tech = df.product_tech_spec[i]
        y = len(product_tech)  
        for j in range(y):
            new_columns.append(product_tech[j]['info'])
    if df.product_addl_info[i] is not None:
        product_add = df.product_addl_info[i]
        z = len(product_add)  
        for j in range(z):
            new_columns.append(product_add[j]['info'])

In [8]:
for i in range(len(new_columns)):
    if new_columns[i] is not None:
        new_columns[i] = new_columns[i].lower()
new_columns = set(new_columns) 

In [9]:
new_columns = list(new_columns)

In [10]:
try:
    new_columns.remove(None)
except:
    pass

In [11]:
for i in new_columns:
    df[i] = i
    df[i] = ''

In [12]:
new_columns.sort()

In [13]:
len(new_columns)

105

In [14]:
for i in range(len(df)):
    if df.product_summary[i] is not None:
        psc = []
        product_sum = df.product_summary[i]
        product_sum = pd.DataFrame(product_sum)
        product_sum = product_sum.transpose()
        psc_len = len(product_sum.columns)  # collecting number of columns in product_sum / items in df.product_summary
        for j in range(psc_len):            # getting item / column names
            psc.append(product_sum[j][0])
        for k in range(len(psc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if psc[k] is not None:
                psc[k] = psc[k].lower()
        product_sum.columns = product_sum.iloc[0] # Turns row 1 into column names
        product_sum = product_sum.drop('info') # Drops row 1
        product_sum.columns = psc
        for l in psc: 
            try:
                df[l][i] = product_sum[l]['value']
            except:
                continue
    if df.product_tech_spec[i] is not None:
        ptc = []
        product_tec = df.product_tech_spec[i]
        product_tec = pd.DataFrame(product_tec)
        product_tec = product_tec.transpose()
        ptc_len = len(product_tec.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(ptc_len):            # getting item / column names
            ptc.append(product_tec[j][0])
        for k in range(len(ptc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if ptc[k] is not None:
                ptc[k] = ptc[k].lower()
        product_tec.columns = product_tec.iloc[0] # Turns row 1 into column names
        product_tec = product_tec.drop('info') # Drops row 1
        product_tec.columns = ptc
        for l in ptc:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_tec[l]['value']
            except:
                continue
    if df.product_addl_info[i] is not None:
        pac = []
        product_add = df.product_addl_info[i]
        product_add = pd.DataFrame(product_add)
        product_add = product_add.transpose()
        pac_len = len(product_add.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(pac_len):            # getting item / column names
            pac.append(product_add[j][0])
        for k in range(len(pac)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if pac[k] is not None:
                pac[k] = pac[k].lower()
        product_add.columns = product_add.iloc[0] # Turns row 1 into column names
        product_add = product_add.drop('info') # Drops row 1
        product_add.columns = pac
        for l in pac:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_add[l]['value']
            except:
                continue

In [15]:
df.rename(columns = {'asin' : 'ASIN'}, inplace = True)

In [18]:
df.drop_duplicates(subset ="ASIN", keep = False, inplace = True) 

In [26]:
df['customer reviews']

0       4.9
1       4.8
3       4.9
5       4.8
7       4.8
       ... 
6097    NaN
6098    NaN
6103    3.0
6105    5.0
6107    NaN
Name: customer reviews, Length: 2477, dtype: object

In [21]:
df['customer reviews'] = df['customer reviews'].str.split('{\ntext-decoration: none;\n} ').str[1]
df['customer reviews'] = df['customer reviews'].str.split(' out of 5 stars').str[0]
# df['customer reviews'] = df['customer reviews'].str.split(' out of 5 stars').str[0]
# df['customer reviews'] = df['customer reviews'].str.split(' von 5 Sternen').str[0]

# df['ASIN'] = df.url.str.split('/dp/').str[-1]
# df['ASIN'] = df.ASIN.str.split('%2Fdp%2F').str[-1]
# df['ASIN'] = df.ASIN.str.split('/').str[0]
# df['ASIN'] = df.ASIN.str.split('%2F').str[0]

In [25]:
df['customer reviews'] = df['customer reviews'].str.replace(',','.')

In [40]:
# Solved above differently because re-scraping resolved in different data basis

for i in range(len(df)):
#     if df['customer reviews'][i] is not None:
#         try:
#             res = [re.findall(r'(\d+).(\d+) out of 5 stars', df['customer reviews'][i])[0] ]
#             a = res[0][0]
#             b = res[0][1]
#             c = float(a+'.'+b)
#             df['customer reviews'][i] = c
#         except:
#             pass
    #  else:
    #      df['customer reviews'][i] = 0
     if df['customer reviews'][i] is None:
         df['customer reviews'][i] = 0

KeyError: 'customer reviews'

In [27]:
df.rename(columns = {'customer reviews' : 'rating'}, inplace = True)

In [28]:
df.rating

0       4.9
1       4.8
3       4.9
5       4.8
7       4.8
       ... 
6097    NaN
6098    NaN
6103    3.0
6105    5.0
6107    NaN
Name: rating, Length: 2477, dtype: object

In [29]:
df = df.drop(['product_summary', 'product_tech_spec', 'product_addl_info'], axis = 1)

In [31]:
df.reset_index(drop=True, inplace = True)

In [32]:
for i in range(len(df)):
    if df.name[i] is None:
        df = df.drop([i], axis = 0)

In [33]:
# Creating dataframe with links to all customer reviews for saving in additional CSV-file

df_processor_customer_reviews_link = df.link_to_all_reviews
df_processor_customer_reviews_link = df_processor_customer_reviews_link.dropna()
df_processor_customer_reviews_link.reset_index(drop=True, inplace = True)

In [34]:
df_processor_bought_together = df[['ASIN', 'name', 'freq_bought', 'freq_bought_link']]

In [35]:
df.to_csv(csv_filename)
#df.to_csv(csv_filename, sep=';')
df_processor_customer_reviews_link.to_csv(csv_customer_reviews) # actually not necessary anymore since there's being a txt-file created
df_processor_bought_together.to_csv(csv_bought_together)

In [36]:
f = open(txt_customer_reviews, 'w+')

for i in range(len(df_processor_customer_reviews_link)):
    f.write(df_processor_customer_reviews_link[i])
    f.write('\n')

f.close()

In [37]:
df2 = pd.read_csv(csv_filename)
df2

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,phone talk time,ink color,phone standy time (with data),...,item display height,processor brand,connector type,processor count,installation method,series,ASIN,number of usb 2.0 ports,material type,computer memory type
0,0,"AMD Ryzen 5 3600 6-Core, 12-Thread Unlocked De...",NaN,NaN,NaN,NaN,https://www.amazon.com//AMD-Ryzen-3600-12-Thre...,NaN,NaN,NaN,...,NaN,AMD,NaN,6.0,NaN,Ryzen 5,B07STGGQ18,NaN,NaN,NaN
1,1,Intel Core i9-9900K Desktop Processor 8 Cores ...,Ideabuy,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,https://www.amazon.com//Intel-i9-9900K-Process...,NaN,NaN,NaN,...,NaN,Intel,NaN,1.0,NaN,NaN,B089J731BX,NaN,NaN,NaN
2,2,"AMD Ryzen 7 3700X 8-Core, 16-Thread Unlocked D...",NaN,NaN,NaN,NaN,https://www.amazon.com//AMD-Ryzen-3700X-16-Thr...,NaN,NaN,NaN,...,NaN,AMD,NaN,8.0,NaN,Ryzen 7,B07SXMZLPK,NaN,NaN,NaN
3,3,AMD Ryzen 5 2600 Processor with Wraith Stealth...,NaN,NaN,NaN,NaN,https://www.amazon.com//AMD-Processor-Wraith-S...,NaN,NaN,NaN,...,NaN,AMD,NaN,6.0,NaN,NaN,B07B41WS48,NaN,NaN,DDR4 SDRAM
4,4,Intel Core i7-9700K Desktop Processor 8 Cores ...,Ideabuy,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,https://www.amazon.com//Intel-i7-9700K-Desktop...,NaN,NaN,NaN,...,NaN,Intel,NaN,8.0,NaN,NaN,B07HHN6KBZ,1.0,NaN,DDR4 SDRAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,2472,HP 13.3 EB 830G7 i710810U 16G 51inch,olecto,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Intel,NaN,1.0,NaN,HP EliteBook,B08FTKLPDP,NaN,NaN,DDR4 SDRAM
2471,2473,Dell Latitude 5400 - 14” FHD Non-Touch - Intel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Intel,NaN,4.0,NaN,Dell Latitude,B08N6ZLKL3,NaN,NaN,DDR4 SDRAM
2472,2474,ZTC RAID Series mSATA Mini or Full Size to SAT...,Hot Deals 4 Less®,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,https://www.amazon.com//ZTC-Enclosure-Supports...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,B00UMFVV9C,NaN,NaN,NaN
2473,2475,"Dell Vostro 13 5390, 8th Generation Intel Core...",Sysmel,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,https://www.amazon.com//Dell-Generation-i7-856...,NaN,NaN,NaN,...,NaN,Intel,NaN,4.0,NaN,Vostro 3000,B07SZCVNLN,2.0,NaN,DDR3 SDRAM


In [38]:
df2[df2.rating == 0]

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,phone talk time,ink color,phone standy time (with data),...,item display height,processor brand,connector type,processor count,installation method,series,ASIN,number of usb 2.0 ports,material type,computer memory type


In [43]:
df2.columns

Index(['Unnamed: 0', 'name', 'seller', 'seller_link', 'freq_bought',
       'freq_bought_link', 'link_to_all_reviews', 'manufacturer part number',
       'memory speed', 'processor count', 'voltage', 'card description',
       'screen resolution', 'type of bulb', 'sheet size',
       'average battery life (in hours)', 'colour', 'department', 'language',
       'item thickness', 'style', 'audio jack', 'series', 'brand',
       'brand name', 'additional features', 'item package quantity',
       'finish types', 'material type', 'chipset brand', 'best sellers rank',
       'os', 'batteries required?', 'computer memory type',
       'item model number', 'color', 'operating system',
       'total recycled content percentage', 'weight', 'item weight',
       'warranty description', 'closure', 'import designation', 'ASIN',
       'rating', 'mfg recommended age', 'graphics card ram size',
       'hard drive interface', 'batteries required', 'number of usb 2.0 ports',
       'batteries', 'hardw